# Visualize calculation result in pandas dataframe format
In this part, we could see how this library could also visualize your log from a dataframe format. If you have your well log in an ASCII format other than las format, you could transform it to pandas dataframe and use this library to visualize it. You can use this to visualize the result of your petrophysical calculation, your log normalization or maybe predicted log.

In this example, I used petrophysics code that I already import before. The code was adapted from Andy McDonald (https://github.com/andymcdgeo/Petrophysics-Python-Series)

#### Format for the Pandas dataframe
You need to format your dataframe as displayed below. The most important thing is to keep the depth information as the first column of the dataset. The order of rest of the log is up to you.
<img src="format.png">

#### Import the libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import lasio
import petrophysics #the pre-loaded petrophysics code
from LogViewer import * #import the library

Loading BokehJS ...

#### Load las file

In [2]:
las = lasio.read(r'pux2.las')
wells=lasViewer('pux2.las')
df=las.df()
print (df.columns.values)

#rename column
df=df.rename(columns={'NPHI_LS':'NPHI'})

#drop column which have no record
df_drop=df.dropna()
df_target=df_drop.reset_index()


Log list: ['DEPTH', 'CALI', 'DT', 'GR', 'ILD', 'MSFL', 'NPHI_LS', 'RHOB']
['CALI' 'DT' 'GR' 'ILD' 'MSFL' 'NPHI_LS' 'RHOB']


#### Calculate Petrophysics

In [3]:
well=df_target
#calculate vshale
well['VSHALE']=petrophysics.shale_volume(well['GR'], well['GR'].quantile(q=0.99),
                             well['GR'].quantile(q=0.01))

#calculate Density from Porosity
well['PHI'] = petrophysics.density_porosity(well['RHOB'], 2.65, 1)

#Calculate PHIE
well['PHIECALC'] = well['PHI'] - (well['VSHALE'] * 0.3)

#Calculate RW
well['rw']=petrophysics.rw_hingle(well['PHI'],1, 2, well['ILD'])

#Calculate Archie SW
well['SW'] = petrophysics.sw_archie(well['PHI'], well['ILD'], well['rw'], 1, 2, 2)

#Calculate Simandoux SW
well['SW_SIM'] = petrophysics.sw_simandoux(well['PHIECALC'], well['ILD'], well['rw'], 1, 2, 2, well['VSHALE'],2)

#Limit SW to 1
well['SW_LIM'] = well['SW'].mask(well['SW']>1, 1)
well['SW__SIM_LIM'] = well['SW_SIM'].mask(well['SW_SIM']>1, 1)

In [4]:
well.describe()

,DEPTH,CALI,DT,GR,ILD,MSFL,NPHI,RHOB,VSHALE,PHI,PHIECALC,rw,SW,SW_SIM,SW_LIM,SW__SIM_LIM
count,7928.0000,7928.000000,7928.000000,7928.000000,7928.000000,7928.000000,7928.000000,7928.000000,7928.000000,7928.000000,7928.000000,7928.000000,7.928000e+03,7852.000000,7.928000e+03,7852.000000
mean,2791.7500,13.881994,146.981188,68.214712,0.820573,1.100889,0.417652,2.030256,0.622716,0.375603,0.188788,0.109875,1.000000e+00,1.268312,1.000000e+00,0.950905
std,1144.3804,2.088381,32.813573,11.638761,0.414278,9.015801,0.064787,0.199511,0.221670,0.120916,0.154466,0.056172,4.945613e-17,2.561162,4.945613e-17,0.086843
min,810.0000,5.851000,60.950000,27.483000,0.334000,0.214000,0.158040,1.418000,-0.153100,0.101800,-0.130440,0.013794,1.000000e+00,0.000000,1.000000e+00,0.000000
25%,1800.8750,12.649000,117.050000,62.784000,0.601000,0.628000,0.369870,1.879000,0.519300,0.270300,0.057775,0.071134,1.000000e+00,0.925443,1.000000e+00,0.925443
50%,2791.7500,13.271000,146.700000,69.682000,0.718000,0.752000,0.424830,2.021500,0.650700,0.380900,0.197915,0.103538,1.000000e+00,0.999522,1.000000e+00,0.999522
75%,3782.6250,14.161000,176.199000,76.817000,0.955000,0.987000,0.468150,2.204000,0.786600,0.467300,0.296430,0.138497,1.000000e+00,1.367310,1.000000e+00,1.000000
max,4773.5000,22.867000,204.691000,97.831000,10.091000,528.214000,0.598830,2.482000,1.186800,0.746700,0.742860,0.592925,1.000000e+00,221.967674,1.000000e+00,1.000000


## newplot 
After calculating your petrophysical properties and storing in the specified dataframe format, you could display it using the newplot syntax. In this part, you need to specify the unit of the log you plotted.

In [5]:
fig=wells.newplot(well,'PHIECALC', '%', 'PHI')
show (fig)

#### Merge with the original log from las
Finally we could merge the calculation result with newplot syntax and plot the original log from the las file using addplot syntax.

In [6]:
fig1=wells.addplot('GR', 'CALI')
fig2=wells.addplot('NPHI_LS', 'RHOB')
fig3=wells.addplot('MSFL','ILD')
# fig4=wells.addplot('ILD', 'RHOB')
fig5=wells.newplot(well,'PHIECALC', '%', 'PHI')
fig6=wells.newplot(well,'VSHALE', '%')

fig2.y_range = fig1.y_range
fig3.y_range = fig1.y_range
# fig4.y_range = fig1.y_range
fig5.y_range = fig1.y_range
fig6.y_range = fig1.y_range

# myLog=[fig1, fig2, fig3, fig4, fig5, fig6] #Combine all the column in a list

# p=gridplot([myLog], sizing_mode='stretch_both')
p=gridplot([[fig1, fig2, fig3, fig5, fig6]], sizing_mode='stretch_both') #You could also write this manually like this
show(p)